In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch
from skorch.callbacks import EarlyStopping
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=99, hidden_size1=64,hidden_size2=64, output_size=7):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layer3=nn.Linear(hidden_size2, output_size)

        self.reset_parameters()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x= self.relu(x)
        x=self.layer3(x)
        return x

    def reset_parameters(self):
        for layer in self.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()

In [ ]:
model = NeuralNetwork()

In [ ]:
def restart_nn_model(learning_rate = 0.001):
    """
    
    :param learning_rate: 
    :return: 
    """
    
    model = NeuralNetwork()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def get_nn_model():
    """
    
    :return: 
    """
    return model

In [ ]:
def train_nn_model(trainLoader, learning_rate = 0.001, nbr_epoch = 100):
    """
    
    :param trainLoader: 
    :param learning_rate: 
    :param number_epoch: 
    :return: 
    """
    model = NeuralNetwork()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()

    for epoch in range(nbr_epoch): 
        running_loss = 0.0
        total_train = 0
        correct_train = 0
    
        for inputs, labels in trainLoader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
    
        epoch_loss = running_loss / len(trainLoader)
        epoch_accuracy = 100 * correct_train / total_train
        print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')
        return model

In [ ]:
def test_nn_model(testLoader, model):
    """
    
    :param testLoader: 
    :param model: 
    :return: 
    """
    model.eval()
    total_test = 0
    correct_test = 0
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in testLoader:
            labels = labels.long()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.tolist())
            y_pred.extend(predicted.tolist())
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    
    test_accuracy = 100 * correct_test / total_test
    print(f'Accuracy on test set: {test_accuracy}%')

In [ ]:
def tune_nn_hyperparameters(X_valid, y_valid, input_size = 99):
    #parameters to tune
    param_grid = {
        'module__hidden_size1': [4096,2048],
        'module__hidden_size2': [512],
        'batch_size': [153, 150, 152, 151],
        'optimizer__lr': [0.001]
    }

    model = NeuralNetwork(input_size=input_size, hidden_size1=64, hidden_size2=64, output_size=7)

    
    early_stopping = EarlyStopping(
        monitor='valid_loss',  # Change to 'valid_acc' for accuracy
        threshold=0.0001,       # Define your threshold
        threshold_mode='rel',  # 'rel' for relative, 'abs' for absolute
        patience=5            # Number of epochs to wait after condition is met
    )


    # Convert the PyTorch model to a skorch classifier to use in GridSearchCV
    classifier = NeuralNetClassifier(
        model,
        criterion=nn.CrossEntropyLoss,
        optimizer=optim.Adam,
        max_epochs=50, # or choose an appropriate number of epochs
        callbacks=[early_stopping]
    )

    # Use GridSearchCV for hyperparameter tuning, cv for the number of folds in cross-validation, verbose for the explicit stage of tuning
    grid_search = GridSearchCV(classifier, param_grid, scoring='accuracy', cv=3, verbose=1)
    #get grid result
    grid_result= grid_search.fit(X_valid, y_valid)

    # Get the best hyperparameters
    best_hyperparams = grid_search.best_params_


    #get best score
    best_score=grid_search.best_score_

    return best_hyperparams, best_score
